In [ ]:
%pip install -q transformers
%pip install -q git+https://github.com/cthiounn/dalle-tiny.git@fr_encoder
%pip install -q wandb
%pip install --upgrade torch

In [ ]:
WANDB_KEY=""
S3_BUCKET="cthiounn2"
CUSTOM_SAVE_FILE_NAME="fr_encoder"

In [ ]:
!wandb login $WANDB_KEY

In [ ]:
import wandb

config = {
  "learning_rate": 1e-5,
  "epochs": 200,
  "batch_size": 20
}
# wandb.init(project="my-test-project", entity="cthiounn",id="1piyh3bl",resume="must")
wandb.init(project="dalle-tiny", entity="cthiounn",config=config)


In [ ]:
from tqdm import tqdm
import s3fs
import os

S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

def write_file_to_s3(bucket_name:str,dir_file:str,file_name:str,fs:s3fs.core.S3FileSystem):
    if bucket_name and file_name and fs:
        FILE_PATH_OUT_S3 = bucket_name + "/" + file_name
        with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out , open(dir_file+file_name, 'rb') as file_in:
            file_out.write(file_in.read())
            
files=['config.json','pytorch_model.bin']
for file in tqdm(files):
    with fs.open(f'{S3_BUCKET}/{file}', mode="rb") as file_in, open(file,"wb") as file_out:
            file_out.write(file_in.read())


In [ ]:
from dalle_tiny.model import TinyDalleModel
from dalle_tiny.util import TinyDalleDataset
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.manual_seed_all(42)
torch.manual_seed(42)

training_data = TinyDalleDataset(parquet_file="https://github.com/cthiounn/dalle-tiny/raw/main/archive_train.parquet",dataset_type="train")
test_data = TinyDalleDataset(parquet_file="https://github.com/cthiounn/dalle-tiny/raw/main/archive_val.parquet",dataset_type="val")

import torch.utils.data as data_utils

indices = torch.arange(12000)
tr_12k = data_utils.Subset(training_data, indices)

indices = torch.arange(480)
te_10k = data_utils.Subset(test_data, indices)
#12000
#495
train_dataloader = DataLoader(tr_12k, batch_size=config["batch_size"], shuffle=True,pin_memory=True)
test_dataloader = DataLoader(te_10k, batch_size=config["batch_size"],pin_memory=True)


In [ ]:
from tqdm import tqdm
from transformers import BartForConditionalGeneration
import gc

import torch.nn as nn

torch.backends.cudnn.benchmark = True

def freeze_params(model):
    for par in model.parameters():
        par.requires_grad = False

#del model
gc.collect()
torch.cuda.empty_cache()

try :
    model=TinyDalleModel.from_pretrained('.')
except:
    model=TinyDalleModel.from_pretrained('facebook/bart-large-cnn')

model.reinit_model_for_images()

freeze_params(model.get_encoder())
freeze_params(model.get_decoder())
model.lm_head=nn.Linear(in_features=1024, out_features=16384+1, bias=False)
model.final_logits_bias=torch.rand(16384+1)    

if torch.cuda.device_count() > 1:
  model = nn.DataParallel(model)

model=model.to(device)

optimizer = optim.AdamW(model.parameters(), betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01, lr=config["learning_rate"])

loss_function = nn.CrossEntropyLoss()

def loss_fn(logits, labels):
    batch_size=logits.shape[0]
    seq_size=logits.shape[1]
    embed_size=logits.shape[2]
    return loss_function(logits.reshape((batch_size*seq_size,embed_size)), labels.reshape(batch_size*seq_size))



num_batches_test = len(test_dataloader)

wandb.watch(model)

i=0
for epoch in range(1,201):
    model.train()
    for batch in tqdm(train_dataloader):
        caption,label =batch
        inp=caption.to(device)
        lab=label[0].to(device)
        shifted_label = lab.new_zeros(lab.shape)
        shifted_label[:, 1:] = lab[:, :-1].clone()
        shifted_label[:, 0] = 16384
        shifted_label=shifted_label.to(device)

        predict=model(input_ids=inp, decoder_input_ids =shifted_label)
        
        
        loss = loss_fn(predict.logits,lab)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        i+=1
        if i%100==0:
            wandb.log({"train_loss": loss.item()})
            print(f"train_loss:{loss.item()}")
        
        
        del inp, lab, predict, loss, shifted_label
        torch.cuda.empty_cache()

        

    model.eval()
    test_loss=0
    with torch.no_grad():
        for batch in tqdm(test_dataloader): 
            caption,label =batch
            inp=caption.to(device)
            lab=label[0].to(device)
            shifted_label = lab.new_zeros(lab.shape)
            shifted_label[:, 1:] = lab[:, :-1].clone()
            shifted_label[:, 0] = 16384
            shifted_label=shifted_label.to(device)
            predict=model(input_ids=inp, decoder_input_ids =shifted_label)
            loss = loss_fn(predict.logits, lab)
            test_loss += loss.item()
            del inp, lab, predict, loss, shifted_label
            torch.cuda.empty_cache()
            
        mean_test_loss=test_loss/num_batches_test
        wandb.log({"mean test_loss": mean_test_loss})
        print(f"mean test loss:{mean_test_loss}")
    if epoch % 5 == 0 :
        file_name=f"../../checkpoint_{CUSTOM_SAVE_FILE_NAME}_{epoch}.pth"
        torch.save(model.module.state_dict(),file_name)
        try:
            write_file_to_s3(S3_BUCKET,"../../",f"checkpoint_{CUSTOM_SAVE_FILE_NAME}_{epoch}.pth",fs)
        except:
            print(f"can't write {file_name}")